In [1]:
import sys
sys.path.append( "/home/mathia/PycharmProjects/neslearn-git")
sys.path.append( "/home/mathia/PycharmProjects/neslearn-git/extern/nes-async-rl/")

In [2]:
import pandas
import hsa.ba.rewards as rewards
import itertools
from extern.fceux_learningenv.nes_python_interface.nes_python_interface import NESInterface
import hsa.machine_constants
from hsa.nes_python_input import py_to_nes_wrapper

import numpy as np
import matplotlib.pyplot as plt
import itertools
import os.path

In [3]:
from dqn_phi import dqn_phi
import nes 
from a3c_nes import A3CLSTM
import chainer
from chainer import serializers

In [7]:
example_model_path = "/home/mathia/checkpoints/scaled_fine_main_reward_1/16000000_finish.h5" 
rom_path = "/home/mathia/mario.nes"
nr_of_attempts = 30

In [5]:
base_paht = "/home/mathia/checkpoints/{}/{}"
model_names = [
    "scaled_fine_main_reward_1"
    ,"nes_a3c_4"
    ,"nes_a3c_scaled_finer_main_reward_1"
    ,"scaled_finer_main_reward_with_points_1"
]
model_chepoint_combos = [(model_name,"16000000_finish.h5")for model_name in model_names]
# model_chepoint_combos.append(("nes_a3c_4","9000000.h5"))
# model_chepoint_combos.append(("scaled_fine_main_reward_1","9400000.h5"))
# model_chepoint_combos.append(("nes_a3c_scaled_finer_main_reward_1","9200000.h5"))
#model_chepoint_combos = itertools.product(model_names, chepoint_filenames) 
model_snapshots = {(directory,checkpoint):base_paht.format(directory,checkpoint) 
                   for (directory,checkpoint) 
                   in model_chepoint_combos}
# make sure every file exists
for path in model_snapshots.values():
    if not os.path.isfile(path):
        print(path)

In [6]:
model_snapshots

{('nes_a3c_4',
  '16000000_finish.h5'): '/home/mathia/checkpoints/nes_a3c_4/16000000_finish.h5',
 ('nes_a3c_scaled_finer_main_reward_1',
  '16000000_finish.h5'): '/home/mathia/checkpoints/nes_a3c_scaled_finer_main_reward_1/16000000_finish.h5',
 ('scaled_fine_main_reward_1',
  '16000000_finish.h5'): '/home/mathia/checkpoints/scaled_fine_main_reward_1/16000000_finish.h5',
 ('scaled_finer_main_reward_with_points_1',
  '16000000_finish.h5'): '/home/mathia/checkpoints/scaled_finer_main_reward_with_points_1/16000000_finish.h5'}

In [8]:
#n_actions = nes.NES(hsa.machine_constants.mario_rom_location).number_of_actions

def load_model(weights_path):
    n_actions = nes.n_actions
    model = A3CLSTM(n_actions)
    serializers.load_hdf5(weights_path, model)
    return model
example_model = load_model(example_model_path)

In [9]:
def make_eval_functions():
    return {
        "main_reward": rewards.make_finer_main_reward(),
        "points": rewards.make_delta_points(),
        "time_left": rewards.reward_for_time_left
    }

In [10]:
def eval_performance(rom, model, deterministic=False,
                     record_screen_path=None):
    nesApi = NESInterface(rom)
    env = nes.NES(rom,outside_nes_interface=nesApi,record_screen_path=record_screen_path)
    model.reset_state()
    ehrenbrav_sum = 0
    other_rewards = make_eval_functions()
    sums = {name:0 for name in other_rewards.keys()}
    while not env.is_terminal:
        s = chainer.Variable(np.expand_dims(dqn_phi(env.state), 0))
        pout = model.pi_and_v(s)[0]
        model.unchain_backward()
        if deterministic:
            a = pout.most_probable_actions[0]
        else:
            a = pout.action_indices[0]
        ehrenbrav_sum += env.receive_action(a)
        
        # TODO Sum of Ehrenbrav, Sum of Main Reward, sum of time left, did it beat the level?, how many deaths
        ram = nesApi.getRAM()
        for name, reward_function in other_rewards.items():
            sums[name] += reward_function(ram)
    
    sums["ehrenbrav"] = ehrenbrav_sum
    return sums

In [38]:
eval_performance(hsa.machine_constants.mario_rom_location,example_model,deterministic=False)

{'ehrenbrav': 14405, 'main_reward': 7.890625, 'points': 40.0, 'time_left': 0}

In [11]:
eval_results_d = {}
for (experiment_name,model_name), weights in model_snapshots.items():
    model = load_model(weights)
    for attempt_nr in range(nr_of_attempts):
        performance = eval_performance(rom_path,model,deterministic=True)
        print((experiment_name,model_name,attempt_nr), performance)
        eval_results_d[(experiment_name,model_name,attempt_nr)] = performance

('nes_a3c_4', '16000000_finish.h5', 0) {'ehrenbrav': 15565, 'points': 50.0, 'main_reward': 4.859375, 'time_left': 0}


('nes_a3c_4', '16000000_finish.h5', 1) {'ehrenbrav': 12140, 'points': 50.0, 'main_reward': 2.6328125, 'time_left': 0}


('nes_a3c_4', '16000000_finish.h5', 2) {'ehrenbrav': 15565, 'points': 50.0, 'main_reward': 4.859375, 'time_left': 0}


('nes_a3c_4', '16000000_finish.h5', 3) {'ehrenbrav': 15565, 'points': 50.0, 'main_reward': 4.859375, 'time_left': 0}


('nes_a3c_4', '16000000_finish.h5', 4) {'ehrenbrav': 12135, 'points': 50.0, 'main_reward': 2.6328125, 'time_left': 0}


...

('scaled_fine_main_reward_1', '16000000_finish.h5', 27) {'ehrenbrav': 16870, 'points': 50.0, 'main_reward': 6.0625, 'time_left': 0}


('scaled_fine_main_reward_1', '16000000_finish.h5', 28) {'ehrenbrav': 16870, 'points': 50.0, 'main_reward': 6.0625, 'time_left': 0}


('scaled_fine_main_reward_1', '16000000_finish.h5', 29) {'ehrenbrav': 16870, 'points': 50.0, 'main_reward': 6.0625, 'time_left': 0}


('nes_a3c_scaled_finer_main_reward_1', '16000000_finish.h5', 0) {'ehrenbrav': 30540, 'points': 70.0, 'main_reward': 11.4453125, 'time_left': 0}


('nes_a3c_scaled_finer_main_reward_1', '16000000_finish.h5', 1) {'ehrenbrav': 30540, 'points': 70.0, 'main_reward': 11.4453125, 'time_left': 0}


In [ ]:
result_d_df_partial = pandas.DataFrame.from_dict(eval_results_d,orient="index")

In [15]:
result_df.to_csv("eval_results_stocastisch.csv")

In [16]:
result_df.to_pickle("eval_results_stocastisch.pickle")

In [19]:
result_df.groupby?